**AutoML OSS入門（8）**
# データ型ベースのディープラーニング AutoML OSS「Ludwig」

本ノートブックの紹介記事と併せてご覧ください。
- [＠IT連載 AutoML OSS入門（8）- 第8回「データ型ベースのディープラーニングAutoML OSS「Ludwig」」](https://atmarkit.itmedia.co.jp/ait/articles/2112/21/news003.html)

また、ノートブックの扱い方やタイタニックの生存予測データについては、連載記事の第1回も参照してください。
- [＠IT連載 AutoML OSS入門（1）- 第1回「機械学習モデル構築作業の煩雑さを解消する「AutoML」とは――歴史、動向、利用のメリットを整理する」](https://www.atmarkit.co.jp/ait/articles/2107/02/news006.html)

## タイタニックデータでAutoML
タイタニックの生存予測データを使って、Ludwigを紹介していきます。

### セットアップ
Ludwigをインストールします。データの読み込みに使うライブラリであるPetastormも併せてインストールしておきます。

In [ ]:
# Ludwigのインストール
!pip install ludwig

# Petastormのインストール
!pip install petastorm

     |████████████████████████████████| 342 kB 18.0 MB/s 
     |████████████████████████████████| 9.5 MB 35.3 MB/s 
     |████████████████████████████████| 1.1 MB 64.2 MB/s 
     |████████████████████████████████| 132 kB 57.1 MB/s 
     |████████████████████████████████| 47 kB 4.9 MB/s 
  Created wheel for ludwig: filename=ludwig-0.4-py3-none-any.whl size=423616 sha256=bfbabb412a247cc9e0373397f9edcc3f19b29da3b1896b87416e1bc5736016bb
  Stored in directory: /root/.cache/pip/wheels/26/3f/00/ccf96056909f13add23251dd78ddfab5e1621125d249947bad
Successfully built ludwig
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5


     |████████████████████████████████| 283 kB 28.9 MB/s 
     |████████████████████████████████| 44 kB 2.7 MB/s 
     |████████████████████████████████| 281.3 MB 40 kB/s 
     |████████████████████████████████| 198 kB 56.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=b15b9dcb51fbf664aa991165d200546d7a534bf48436c3be342f2335b37c4b8e
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [ ]:
import ludwig
ludwig.__version__

'0.4'

### ライブラリーのインポート

In [ ]:
from ludwig.api import LudwigModel

### データのダウンロード
タイタニックの生存予測データをダウンロードします。<br>
第1回「AutoMLとは」の「タイタニックの生存予測データの取得方法と解説」の章を参照してください。

In [ ]:
# データの準備
!wget -N https://github.com/aiq2020-tw/automl-notebooks/raw/main/titanic.zip
!unzip titanic.zip

A
--2021-11-23 07:00:48--  https://github.com/aiq2020-tw/automl-notebooks/raw/main/titanic.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/aiq2020-tw/automl-notebooks/main/titanic.zip [following]
--2021-11-23 07:00:48--  https://raw.githubusercontent.com/aiq2020-tw/automl-notebooks/main/titanic.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34877 (34K) [application/zip]
Saving to: ‘titanic.zip’

titanic.zip         100%[===================>]  34.06K  --.-KB/s    in 0s      

Last-modified header missing -- time-stamps turned off.
2021-11-23 07:00:48 (126 MB/s) - ‘titanic.zip’ saved [348

In [ ]:
# 学習データとテストデータの確認
import pandas as pd
train_df = pd.read_csv('train.csv')
print('Train data shape:', train_df.shape)
test_df = pd.read_csv('test.csv')
print('Test data shape:', test_df.shape)

Train data shape: (891, 12)
Test data shape: (418, 11)


### データの前処理
前処理などの必要な情報は、configとして事前に定義しておきます。<br>
入力データ列と出力データ列にも定義しない列は利用されないため、元のテーブルに対して列を削除しておくような前処理は不要です。

In [ ]:
# 辞書型でconfigを記載
config = {
    # データ分割の比率を設定する
    'preprocessing': {'split_probabilities': [0.7, 0.2, 0.1]},
    # 説明変数を設定する
    'input_features': [{'name': 'Pclass', 'type': 'category'},
                       {'name': 'Sex', 'type': 'category'},
                       {'name': 'Age',
                        'preprocessing': {
                            'missing_value_strategy': 'fill_with_mean'},
                        'type': 'numerical'},
                       {'name': 'SibSp', 'type': 'numerical'},
                       {'name': 'Parch', 'type': 'numerical'},
                       {'name': 'Fare',
                        'preprocessing': {
                            'missing_value_strategy': 'fill_with_mean'},
                        'type': 'numerical'},
                       {'name': 'Embarked', 'type': 'category'}],
    # 目的変数を設定する
    'output_features': [{'name': 'Survived', 'type': 'binary'}],
    # 学習に関するパラメーターを設定する（任意）
    'training': {
        'batch_size': 128,
        'epochs': 300,
        'early_stop': 5,
        'learning_rate': 0.001}
    }

### モデルの学習

In [ ]:
# Ludwigモデルのインスタンスを生成する
import logging
model = LudwigModel(config, logging_level=logging.INFO)

In [ ]:
# モデルの学習を行う
train_stats = model.train(train_df, random_seed=42)

ストリーミング出力は最後の 5000 行に切り捨てられました。
Evaluation test : 100%|██████████| 1/1 [00:00<00:00, 425.69it/s]
Took 0.1289s
╒════════════╤════════╤════════════╕
│ Survived   │   loss │   accuracy │
╞════════════╪════════╪════════════╡
│ train      │ 0.6830 │     0.6270 │
├────────────┼────────┼────────────┤
│ vali       │ 0.6398 │     0.6686 │
├────────────┼────────┼────────────┤
│ test       │ 0.6541 │     0.6739 │
╘════════════╧════════╧════════════╛
╒════════════╤════════╕
│ combined   │   loss │
╞════════════╪════════╡
│ train      │ 0.6830 │
├────────────┼────────┤
│ vali       │ 0.6398 │
├────────────┼────────┤
│ test       │ 0.6541 │
╘════════════╧════════╛
Validation loss on combined improved, model saved


Epoch  41
Evaluation test : 100%|██████████| 1/1 [00:00<00:00, 338.99it/s]
Took 0.1363s
╒════════════╤════════╤════════════╕
│ Survived   │   loss │   accuracy │
╞════════════╪════════╪════════════╡
│ train      │ 0.6777 │     0.6333 │
├────────────┼────────┼────────────┤
│ vali       │ 0

### テストデータの予測
学習したモデルを用いてテストデータの予測を行います。<br>
分類の予測結果（Survived_predictions）だけでなく、その確率（Survived_probability）も併せて出力されます。

In [ ]:
# 予測
predictions = model.predict(test_df)
predictions[0]

Prediction: 100%|██████████| 4/4 [00:00<00:00, 13.17it/s]


,Survived_probabilities,Survived_predictions,Survived_probabilities_False,Survived_probabilities_True,Survived_probability
0,"[0.8923938572406769, 0.10760614275932312]",False,0.892394,0.107606,0.892394
1,"[0.509885311126709, 0.490114688873291]",False,0.509885,0.490115,0.509885
2,"[0.8041219711303711, 0.1958780288696289]",False,0.804122,0.195878,0.804122
3,"[0.8927166759967804, 0.1072833240032196]",False,0.892717,0.107283,0.892717
4,"[0.4722605347633362, 0.5277394652366638]",True,0.472261,0.527739,0.527739
...,...,...,...,...,...
413,"[0.8951968252658844, 0.1048031747341156]",False,0.895197,0.104803,0.895197
414,"[0.05979800224304199, 0.940201997756958]",True,0.059798,0.940202,0.940202
415,"[0.9023429751396179, 0.09765702486038208]",False,0.902343,0.097657,0.902343
416,"[0.8951968252658844, 0.1048031747341156]",False,0.895197,0.104803,0.895197


In [ ]:
# Kaggle投稿用のファイルを作成する
output = pd.DataFrame({'PassengerID': test_df.PassengerId,
                       'Survived':
                       predictions[0]['Survived_predictions'].astype('int32')})
output.to_csv('submission.csv', index=False)

## その他の機能と応用

### ハイパーパラメーターチューニング
ludwigのサブパッケージである「hyperopt」を用いてハイパーパラメーターチューニングを行います。<br>
インストール後、必要に応じてランタイムの再起動を実行してください。

In [ ]:
# ハイパーパラメーターチューニング用のライブラリを別にインストール
!pip install ludwig[hyperopt]

# scikit-learnのアップグレード
!pip install --upgrade scikit-learn==0.22.2.post1

     |████████████████████████████████| 61 kB 191 kB/s 
     |████████████████████████████████| 54.7 MB 1.3 MB/s 
     |████████████████████████████████| 86 kB 5.0 MB/s 
     |████████████████████████████████| 72 kB 1.2 MB/s 
     |████████████████████████████████| 180 kB 54.3 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 165 kB 20.2 MB/s 
     |████████████████████████████████| 146 kB 52.7 MB/s 
     |████████████████████████████████| 1.7 MB 54.7 MB/s 
     |████████████████████████████████| 52 kB 1.4 MB/s 
     |████████████████████████████████| 596 kB 53.5 MB/s 
     |████████████████████████████████| 119 kB 52.8 MB/s 
     |████████████████████████████████| 124 kB 58.0 MB/s 
  Created wheel for bayesmark: filename=bayesmark-0.0.8-py3-none-any.whl size=104281 sha256=8f6762f580ee703ac2b0dd1e860ac43d1a03d9659dd920966fa8d93262b6c98e
  Stored in directory: /root/.cache/pip/wheels/2f/ce/4c/a5b1d0c4df6a56f23176d0100727f91d353e625bbd0

     |████████████████████████████████| 7.1 MB 32.5 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.1
    Uninstalling scikit-learn-1.0.1:
      Successfully uninstalled scikit-learn-1.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imbalanced-learn 0.8.1 requires scikit-learn>=0.24, but you have scikit-learn 0.22.2.post1 which is incompatible.


In [ ]:
# ハイパーパラメーターチューニングの探索範囲を定義
hyperopt_configs = {
    'parameters': {
        'training.learning_rate': {
            'type': 'float',
            'low': 0.0001,
            'high': 0.01,
            'space': 'log',
            'steps': 3,
            },
        'training.batch_size': {
            'type': 'int',
            'low': 32,
            'high': 256,
            'space': 'log',
            'steps': 5,
            'base': 2
            }
        },
    'output_feature': 'Survived',
    'metrics': 'loss',
    'goal': 'minimize',
    'sampler': {
        'type': 'random',
        'num_samples': 10
        },
    'executor': {
        'type': 'parallel',
        'num_workers': 4
        }
    }

# configに対して上記の設定を追加
config['hyperopt'] = hyperopt_configs

In [ ]:
# ハイパーパラメーターチューニングの実行
from ludwig.hyperopt.run import hyperopt
random_parallel_results = hyperopt(
    config,
    dataset=train_df,
    output_directory='results_random_parallel'
)

{   'executor': {   'TF_REQUIRED_MEMORY_PER_WORKER': 100,
                    'epsilon': 0.01,
                    'epsilon_memory': 100,
                    'num_workers': 4,
                    'type': 'parallel'},
    'goal': 'minimize',
    'metric': 'loss',
    'metrics': 'loss',
    'output_feature': 'Survived',
    'parameters': {   'training.batch_size': {   'base': 2,
                                                 'high': 256,
                                                 'low': 32,
                                                 'space': 'log',
                                                 'steps': 5,
                                                 'type': 'int'},
                      'training.learning_rate': {   'high': 0.01,
                                                    'low': 0.0001,
                                                    'space': 'log',
                                                    'steps': 3,
                                          

ハイパーパラメーターチューニングの結果、予測精度の高いパラメーターを発見できました。

以上で、タイタニックの生存予測データを使ったLudwigの紹介は終わりです。